In [ ]:
%load_ext pycodestyle_magic
# %%pycodestyle 사용으로 코드 검사

# 정규표현식
import re
# html 받기
import requests
# from urllib import urlopen >> cannot import name 'urlopen' from 'urllib' (C:\Users\ChoSungHoon\Anaconda3\lib\urllib\__init__.py)
from urllib.request import urlopen
# html 파싱
from bs4 import BeautifulSoup as bs
from time import sleep
# 크롬 드라이버 사용
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
# 자신의 크롬버전에 맞는 드라이버 사용
driver = webdriver.Chrome("chromedriver")
driver.get("https://www.siksinhot.com/search?keywords=%EC%9D%B8%EC%B2%9C")
#driver.get("https://www.siksinhot.com/search?keywords=%EC%84%9C%EC%9A%B8")

In [ ]:

# 더보기 실행으로 #a태그로 못가져오는 모든 음식점 정보 띄우기
while True:
    while True:
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, '더보기'))
        )
        button.click()

In [ ]:

# 크롬 드라이버에서 html 코드 가져오기
html = driver.page_source
# 가져온 코드 파싱
soup = bs(html)

# 각 음식점 페이지에 들어갈 수 있게 해당 주소 저장
restaurant_url = []
get_url = soup.select("#schMove1 > div.listTy1")[0].find_all("a")


for url in get_url:
    #print(url)
    if 'href' in url.attrs:
        if 'P/' in url.attrs["href"]:
            restaurant_url.append(url.attrs["href"])
            


In [ ]:
# python에서 mysql 사용하기
import pymysql
# 가져온 db dataframe으로 사용하기
import pandas as pd

In [ ]:

"""
pymysql.connect() 메소드를 사용하여 MySQL에 연결
호스트명, 포트, 로그인, 암호, 접속할 DB 등을 파라미터로 지정
주요 파라미터
host : 접속할 mysql server 주소
port : 접속할 mysql server 의 포트 번호
user : mysql ID
passwd : mysql ID의 암호
db : 접속할 데이터베이스
charset='utf8' : mysql에서 select하여 데이타를 가져올 때 한글이 깨질 수 있으므로 연결 설정에 넣어줌
"""

# cursorclass=pymysql.cursors.DictCursor >> mysql to dataframe
db = pymysql.connect(host="localhost", port=3306, user="root",
                     password="0000", db="schema_name", charset='utf8', 
                     cursorclass=pymysql.cursors.DictCursor)


In [ ]:
# db 연결한 것을 커서로 이용
cursor = db.cursor()

In [ ]:
# utf8 로 한글 깨지는 것을 다 못막아서 해당 명령어로 해결
cursor.execute("set names utf8mb4")

In [ ]:
# table 없을 때 만들기용으로 실행

sql = """
    CREATE TABLE review_table( review_id VARCHAR(15) NOT NULL,
        restaurant_id VARCHAR(10) NOT NULL,
        writer_id VARCHAR(45) NOT NULL,
        review_contents VARCHAR(10000) NOT NULL,
        review_score VARCHAR(10) NOT NULL,
        PRIMARY KEY(review_id)
    );
    """

cursor.execute(sql)
db.commit()

In [ ]:
# table 없을 때 만들기용으로 실행

sql = """
    CREATE TABLE restaurant_table( restaurant_id VARCHAR(10),
        restaurant_name VARCHAR(100) NOT NULL,
        web_link VARCHAR(100) NOT NULL,
        category VARCHAR(100) NOT NULL,
        phone_number VARCHAR(20) NOT NULL,
        rating VARCHAR(10) NOT NULL,
        location VARCHAR(100) NOT NULL,
        gps_N VARCHAR(20) NOT NULL,
        gps_E VARCHAR(20) NOT NULL,
        image_src VARCHAR(100) NOT NULL,
        main_menu VARCHAR(500) NOT NULL,
        main_menu_price VARCHAR(1000) NOT NULL,
        review_count_number VARCHAR(20) NOT NULL,
        mean_price VARCHAR(20) NOT NULL,
        PRIMARY KEY(restaurant_id)
    );
    """

cursor.execute(sql)
db.commit()

In [ ]:
# restaurant_table 에서 primary key 컬럼을 리스트로 쓰기
sql = "select * from restaurant_table"
cursor.execute(sql)
result = cursor.fetchall()
df_restaurant = pd.DataFrame(result)

restaurnat_id_list = df_restaurant["restaurant_id"].tolist()

In [ ]:
# restaurant_table 에서 primary key 컬럼을 리스트로 쓰기
sql = "select * from review_table"
cursor.execute(sql)
result = cursor.fetchall()
df_review = pd.DataFrame(result)

review_id_list = df_review["review_id"].tolist()

In [ ]:
# 메인 크롤링


for current_url in restaurant_url:
    
    url = "https://www.siksinhot.com"+str(current_url)
    
    current_url=str(current_url)[3:]
    
    ############
    #주소 가져오기
    ############
    
    response = requests.get(url)
    
    if response.status_code == 200:
        html = bs(response.text, "html.parser")
        
        
        ############
        #지도 주소
        ############
        
        mapUrl = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(1) > div:nth-child(6) > p > a.btnTy1.btn_google_map")
        
        if not mapUrl:
            mapUrl = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(1) > div:nth-child(5) > p > a.btnTy1.btn_google_map")
        else:
            pass
        
        if not mapUrl:
            mapUrl = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(1) > div:nth-child(4) > p > a.btnTy1.btn_google_map")
        else:
            pass
        
        mapUrl = mapUrl[0]["href"]
        gps_total = mapUrl.split("=")[1]
        
        gps_n, gps_e = gps_total.split(" , ")
        
        ############
        #음식점 이름
        ############
        
        name = html.select("#contents > div > div > div.title_box > div.title > div.store_name_score > h3")
        
        if not name:
            name=""
            pass
        else:
            name = name[0].getText()
            name = name.split("HOT")[0][0:-3]
            
        if len(name) >= 100:
            name = name[0:100]
        
        ############
        #이미지 가져오기
        ############
        
        img_count = html.select("#contents > div > div > div:nth-child(2) > div > ul > div > div > div")[0].find_all("li")
        
        #이미지 빈거 해결하기 위해 a 갖고 있는것만 카운팅
        #이미지 칸수를 li로 최소 3개를 잡아두어 그냥 카운트하면 원하는 빈 이미지때문에 에러 발생
        img_count2 = 0
        
        for i in range(1,len(img_count)+1):
            if html.select("#contents > div > div > div:nth-child(2) > div > ul > div > div > div > li:nth-child("+str(i)+")")[0].find_all('a'):
                img_count2+=1
        
        img_list=[]
        
        for i in range(1,img_count2+1):
            img_tags = html.select("#contents > div > div > div:nth-child(2) > div > ul > div > div > div > li:nth-child("+str(i)+") > a")[0].find_all('img')
            
            if not len(img_tags):
                pass
            for imgtag in img_tags:
                img_list.append(imgtag['src'])
                
        if not img_list:
            img_list=""
        
        ############
        # 평점
        ############
        
        rating = html.select("#contents > div > div > div.title_box.title_grade > div.title > div > h3 > strong")
        
        if not rating:
            rating = html.select("#contents > div > div > div.title_box > div.title > div.store_name_score > h3 > strong")
            
        if not rating:
            rating=""
            pass
        else:
            rating = rating[0].getText()
        
        ############
        # 전화번호
        ############
        
        phone = html.select(" #contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(1) > div.p_tel > p")
        
        if not phone:
            phone=""
            pass
        else:
            phone = phone[0].getText()
        
        ############
        #홈페이지 #DB에서는 해당 데이터가 너무 적어서 음식점 url 로 대체
        ############
        """
        homePage = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(10) > p > span > a")
        
        if not homePage:
            homePage=""
            pass
        else:
            homePage = homePage[0].getText()
        """
        
        ############
        #업종
        ############
        
        category = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(1) > p")
        
        if not category:
            category=""
            pass
        else:
            category = category[0].getText()
        
        ############
        #주소
        ############
        
        location = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(1) > div:nth-child(5) > p > a.txt_adr")
        
        if not location:
            location = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child(1) > div:nth-child(6) > p > a.txt_adr")
        
        if not location:
            location=""
            pass
        else:
            location = location[0].getText()
        
        #############
        #대표메뉴
        #############
        
        main_menu_list = ""
        main_menu_list_cost = ""
        
        mean_price = 0
        
        for main_menu_cnt in range(0,10):
            main_menu_tag = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child("+str(main_menu_cnt+1)+") > h4")
            
            if not main_menu_tag:
                main_menu_tag=""
                pass
            else:
                main_menu_tag = main_menu_tag[0].getText()
                
            if main_menu_tag=="대표메뉴":
                main_menu_count = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child("+str(main_menu_cnt+1)+") > ul")[0].find_all("li")
                
                if len(main_menu_count) > 1:
                    for main_menu_cnt2 in range(0,len(main_menu_count)):
                        main_menu = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child("+str(main_menu_cnt+1)+") > ul > li:nth-child("+str(main_menu_cnt2+1)+") > span")[0].getText()
                        main_menu_list = main_menu_list + ", " + main_menu
                        main_menu_cost = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child("+str(main_menu_cnt+1)+") > ul > li:nth-child("+str(main_menu_cnt2+1)+") > p > span > em")
                        
                        if not main_menu_cost:
                            main_menu_cost = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child("+str(main_menu_cnt+1)+") > ul > li:nth-child("+str(main_menu_cnt2+1)+") > p > span > em > label")
                        main_menu_cost = main_menu_cost[0].getText()
                        
                        if ' 원' in main_menu_cost:
                            if '~' not in main_menu_cost:
                                tempcost = main_menu_cost[0:-2]
                                if ','in tempcost:
                                    tempf,templ = tempcost.split(',')
                                    mean_price += int(tempf)*1000 + int(templ)
                                else:
                                    mean_price +=int(tempcost)
                            else:
                                mcostf,mcostl=main_menu_cost.split(' ~ ')
                                main_menu_cost = mcostl
                                tempcost = main_menu_cost[0:-2]
                                if ','in tempcost:
                                    tempf,templ = tempcost.split(',')
                                    mean_price += int(tempf)*1000 + int(templ)
                                else:
                                    mean_price +=int(tempcost)
                            
                        
                        main_menu_cost = main_menu + "-- " +main_menu_cost
                        main_menu_list_cost = main_menu_list_cost + ", " + main_menu_cost
                else:
                    main_menu = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child("+str(main_menu_cnt+1)+") > ul > li > span")[0].getText()
                    
                    main_menu_cost = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child("+str(main_menu_cnt+1)+") > ul > li > p > span > em")
                    
                    if not main_menu_cost:
                        main_menu_cost = html.select("#contents > div > div > div.content > div.sec_left > div > div:nth-child(1) > div:nth-child("+str(main_menu_cnt+1)+") > ul > li > p > span > em > label")
                    
                    main_menu_cost = main_menu_cost[0].getText()
                    
                    if ' 원' in main_menu_cost:
                        if '~' not in main_menu_cost:
                            tempcost = main_menu_cost[0:-2]
                            if ',' in tempcost:
                                tempf,templ = tempcost.split(',')
                                mean_price = int(tempf)*1000 + int(templ)
                            else:
                                mean_price = int(tempcost)
                        else:
                            mcostf,mcostl=main_menu_cost.split(' ~ ')
                            main_menu_cost = mcostl
                            tempcost = main_menu_cost[0:-2]
                            if ',' in tempcost:
                                tempf,templ = tempcost.split(',')
                                mean_price = int(tempf)*1000 + int(templ)
                            else:
                                mean_price = int(tempcost)
                    
                    main_menu_list = main_menu_list + ", " + main_menu
                    main_menu_cost = main_menu + "-- " +main_menu_cost
                    main_menu_list_cost = main_menu_list_cost + ", " + main_menu_cost
                    
                mean_price = mean_price/len(main_menu_count)
                mean_price = round(mean_price,0)
                    
                    
                main_menu_list = main_menu_list[2:]
                main_menu_list_cost = main_menu_list_cost[2:]
                mean_price = str(mean_price)[0:-2]
                
                break
        
        ############
        #리뷰
        ############
        
        rewview_count = html.select("#siksin_review > div.txt_total > ul > li:nth-child(1) > span")[0].getText()
        rewview_count = int(rewview_count)
        
        if rewview_count > 5:
            rewview_count = 5
        
        for review_cnt in range(0,rewview_count):
            
            nickname = html.select("#siksin_review > div.rList > ul > li:nth-child("+str(rewview_count+1)+") > div > div.cnt > div.name_data > a > strong")[0].getText()
            
            review_rate = html.select("#siksin_review > div.rList > ul > li:nth-child("+str(rewview_count+1)+") > div > div.cnt > div.score_story > span > strong")
            if not review_rate:
                review_rate =""
            else:
                review_rate = review_rate[0].getText()
            
            review_summary = html.select("#siksin_review > div.rList > ul > li:nth-child("+str(rewview_count+1)+") > div > div.cnt > div.score_story > span")
            if not review_summary:
                review_summary = ""
            else:
                review_summary = review_summary[0].getText()[4:]
            
            reveiw_content = html.select("#siksin_review > div.rList > ul > li:nth-child("+str(review_cnt+1)+") > div > div.cnt > div.score_story > p")
            
            if not reveiw_content:
                reveiw_content=""
            else:
                reveiw_content = reveiw_content[0].getText()
                reveiw_content = str(reveiw_content)
            
            
            if str(current_url)+"_"+str(review_cnt) not in review_id_list:
                sql ="INSERT INTO review_table"+" (review_id, restaurant_id, writer_id, review_contents, review_score) VALUES(%s, %s, %s, %s, %s)"
                val = (str(current_url)+"_"+str(review_cnt),str(current_url),nickname,reveiw_content,review_rate)
                cursor.execute(sql,val)
                db.commit()
            else:
                sql = "UPDATE review_table SET review_id = %s, restaurant_id = %s, writer_id = %s, review_contents = %s, review_score = %s WHERE review_id = %s"
                val = (str(current_url)+"_"+str(review_cnt),str(current_url),nickname,reveiw_content,review_rate,str(current_url)+"_"+str(review_cnt))
                cursor.execute(sql,val)
                db.commit()
        
        review_count = str(rewview_count)
        
        if current_url in restaurnat_id_list:
            sql ="INSERT INTO restaurant_table"+" (restaurant_id, restaurant_name, web_link, category, phone_number, rating, location, gps_N, gps_E, image_src, main_menu, main_menu_price, review_count_number, mean_price) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"   
            val = (str(current_url),name,url,category,phone,rating,location,gps_n,gps_e,img_list[0],main_menu_list,main_menu_list_cost,review_count,mean_price)
            cursor.execute(sql,val)
            db.commit()
        else:
            sql = "UPDATE restaurant_table SET restaurant_id = %s, restaurant_name = %s, web_link = %s, category = %s, phone_number = %s, rating = %s, location = %s, gps_N = %s, gps_E = %s, image_src = %s, main_menu = %s, main_menu_price = %s, review_count_number = %s, mean_price = %s WHERE restaurant_id = %s"
            val = (str(current_url),name,url,category,phone,rating,location,gps_n,gps_e,img_list[0],main_menu_list,mm_list_cost,review_count,mean_price,str(current_url))
            cursor.execute(sql,val)
            db.commit()
        
        
# 삽입, 갱신, 삭제 등이 모두 끝났으면 Connection 객체의 commit() 메서드를 사용하여 데이타를 Commit
db.commit()
# Connection 객체의 close() 메서드를 사용하여 DB 연결을 닫음
db.close()